<a href="https://colab.research.google.com/github/max-enterme/Naga-Calc/blob/main/NAGA%E6%8E%A1%E7%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Feature
- 鳴き・立直判断を考慮したNAGA度の算出

<!--
参考サイト: https://note.com/suoh2009/n/n7eb51e50b4e9

NAGAレポートのデータフォーマットに関するメモ
# 副露の情報: 一個前の打牌情報の "huro" にある
#  - 0: しない
#  - 1: チー (上の両面)
#  - 2: チー (カンチャン)
#  - 3: チー (下の両面)
#  - 4: ポン
#  - 5: 大明槓(?)
# 暗槓の情報: 実行したら、type が "ankan" で入ってくる
# 暗槓するかどうかの判断は、type = tsumo の時に "kan" に入ってくる
-->

In [ ]:
#@title 採点
#@markdown 以下にNAGAレポートのURLを入力し、左部の再生ボタン(▷)を押してください。
url = "" #@param {type:"string"}

import json
import requests
from os import mkdir
from bs4 import BeautifulSoup 
from google.colab import files

def parseNagaReport(url):
  html = requests.get(url)
  soup = BeautifulSoup(html.content, "html.parser")
  script = soup.select("script")[0].text
  scriptPerLine = script.splitlines()
  return {
    "pred": json.loads(scriptPerLine[1].split("=")[1]),
    "playerInfo": json.loads(scriptPerLine[2].split("=")[1].replace("'", "\"")),
    "haihuId": scriptPerLine[3].split("=")[1].strip("  \""),
    "nagaVersion": scriptPerLine[4].split("=")[1].strip("  \""),
    "gameType": scriptPerLine[5].split("=")[1].strip()
  }

pai2index = {
  "1m":0, "2m":1, "3m":2, "4m": 3, "5m":4, "5mr":4, "6m":5, "7m": 6, "8m":7, "9m":8,
  "1p":9, "2p":10, "3p":11, "4p":12, "5p":13, "5pr":13, "6p":14, "7p":15, "8p":16, "9p":17,
  "1s":18, "2s":19, "3s":20, "4s":21, "5s":22, "5sr":22, "6s":23, "7s":24, "8s":25, "9s":26,
  "E":27, "S":28, "W":29, "N":30, "P":31, "F":32, "C":33
}

bakaze_text_map = {
    "E": "東", "S": "南", "W":"西"
}

actors = [0, 1, 2, 3]

def create_actor_report(report, actor):
    return {
        "name": report["playerInfo"]["name"][actor],
        "rank": report["pred"][-1][0]["info"]["msg"]["end_msgs"][0]["seat2rank"][actor],
        "kyokus": [],
        "total_dahai_count": 0,
        "total_matched_dahai_count": 0,
        "total_bad_dahai_count": 0,
        "total_dahai_score_diff": 0,
        "total_dahai_point": 0,
        "total_naga_point": 0
    }

def create_actor_kyoku_info(start_kyoku):
  msg = start_kyoku["info"]["msg"]
  return {
      "bakaze": msg["bakaze"],
      "kyoku": msg["kyoku"],
      "honba": msg["honba"],
      "logs": [],
      "dahai_count": 0,
      "matched_dahai_count": 0,
      "bad_dahai_count": 0,
      "dahai_score_diff": 0,
      "dahai_point": 0,
      "naga_point": 0
  }

def add_action(actor_kyoku_infos, action,):
  msg = action["info"]["msg"]
  if not "actor" in msg:
    return
  actor = msg["actor"]
  # actor_kyoku_infos[actor]["logs"].append(msg)

  if "real_dahai" in msg and msg["real_dahai"] != None and "pred_dahai" in msg and not msg["reached"]:
    dahai_score = action["dahai_pred"][pai2index[msg["real_dahai"]]]
    pred_score = action["dahai_pred"][pai2index[msg["pred_dahai"]]]
    actor_kyoku_infos[actor]["dahai_count"] += 1
    actor_kyoku_infos[actor]["matched_dahai_count"] += 1 if msg["real_dahai"] == msg["pred_dahai"] else 0
    actor_kyoku_infos[actor]["bad_dahai_count"] += 1 if pred_score - dahai_score >= 8500 else 0
    actor_kyoku_infos[actor]["dahai_score_diff"] += pred_score - dahai_score

  # if "kan" in action:
  #   actor_kyoku_infos[actor]["logs"].append(action["kan"])
  # if "huro" in action:
  #   actor_kyoku_infos[actor]["logs"].append(action["huro"])

def read_kyoku_actions(actions):
  actor_kyoku_infos = {}
  global actors
  for actor in actors:
     actor_kyoku_infos[actor] = create_actor_kyoku_info(actions[0])
  for action in actions:
    add_action(actor_kyoku_infos, action)
  for (actor, info) in actor_kyoku_infos.items():
    info["dahai_point"] = 100 - info["dahai_score_diff"] / 100 / info["dahai_count"]
  return actor_kyoku_infos


report = parseNagaReport(url)

actor_reports = {}
for actor in actors:
  actor_reports[actor] = create_actor_report(report, actor)

for actions in report["pred"]:
  for actor, info in read_kyoku_actions(actions).items():
    actor_reports[actor]["kyokus"].append(info)

for actor_report in actor_reports.values():
  for kyoku_info in actor_report["kyokus"]:
    actor_report["total_dahai_count"] += kyoku_info["dahai_count"]
    actor_report["total_matched_dahai_count"] += kyoku_info["matched_dahai_count"]
    actor_report["total_bad_dahai_count"] += kyoku_info["bad_dahai_count"]
    actor_report["total_dahai_score_diff"] += kyoku_info["dahai_score_diff"]
  actor_report["total_dahai_point"] = 100 - actor_report["total_dahai_score_diff"] / 100 / actor_report["total_dahai_count"]

import pandas
import html
from IPython.display import HTML
from urllib.parse import quote

def create_total_result_row(actor_report):
  return [[
      actor_report['name'],
      round(actor_report["total_matched_dahai_count"] / actor_report["total_dahai_count"] * 100,2),
      round(actor_report["total_dahai_point"], 3),
      actor_report['total_matched_dahai_count'],
      actor_report['total_dahai_count'] - actor_report['total_matched_dahai_count'],
      actor_report['total_dahai_count'],
      actor_report['total_bad_dahai_count'],
      actor_report['rank']
  ]]

def create_total_result(actor_reports):
  columns=['名前', '一致率', 'NAGA度', '一致数', '不一致数', '総打数', '悪手数', '順位']
  df = pandas.DataFrame()
  for actor_report in actor_reports.values():
    df = pandas.concat([df, pandas.DataFrame(create_total_result_row(actor_report), columns = columns)])
  df = df.set_index('名前')
  return df

def create_actor_result_row(kyoku_info):
  kyoku_name = f'{bakaze_text_map[kyoku_info["bakaze"]]}{kyoku_info["kyoku"]}局'
  if kyoku_info["honba"] >= 1:
    kyoku_name += f' {kyoku_info["honba"]}本場'
  return [[
      kyoku_name,
      round(kyoku_info["matched_dahai_count"] / kyoku_info["dahai_count"] * 100,2),
      round(kyoku_info["dahai_point"], 3),
      kyoku_info['matched_dahai_count'],
      kyoku_info['dahai_count'] - kyoku_info['matched_dahai_count'],
      kyoku_info['dahai_count'],
      kyoku_info['bad_dahai_count']
  ]]

def create_actor_result(actor_reports, actor):
  global bakaze_text_map
  actor_report = actor_reports[actor]
  columns = ['局', '一致率', 'NAGA度', '一致数', '不一致数', '総打数', '悪手数']
  df = pandas.DataFrame()
  for info in actor_report["kyokus"]:
    df = pandas.concat([df, pandas.DataFrame(create_actor_result_row(info), columns = columns)])   
  df = df.set_index('局')
  return df

In [ ]:
#@title 結果出力 (CSV)
header = False #@param {type:"boolean"}

#@markdown

#@markdown ####パラメータの説明
#@markdown header: 項目名の出力設定

df = create_total_result(actor_reports)
print(df)
clipboard_texts = {
  0: df.iloc[0,:].to_csv(index=header).replace('\n', '\\n'),
  1: df.iloc[1,:].to_csv(index=header).replace('\n', '\\n'),
  2: df.iloc[2,:].to_csv(index=header).replace('\n', '\\n'),
  3: df.iloc[3,:].to_csv(index=header).replace('\n', '\\n'),
  4: df.to_csv(header=header).replace('\n', '\\n'),
  5: df.T.to_csv(index=header).replace('\n', '\\n'),
}
HTML(f"""
<button onclick=navigator.clipboard.writeText('{clipboard_texts[4]}')>Copy All</button>
<button onclick=navigator.clipboard.writeText('{clipboard_texts[5]}')>Copy All T</button>
<button onclick=navigator.clipboard.writeText('{clipboard_texts[0]}')>Copy {actor_reports[0]['name']}</button>
<button onclick=navigator.clipboard.writeText('{clipboard_texts[1]}')>Copy {actor_reports[1]['name']}</button>
<button onclick=navigator.clipboard.writeText('{clipboard_texts[2]}')>Copy {actor_reports[2]['name']}</button>
<button onclick=navigator.clipboard.writeText('{clipboard_texts[3]}')>Copy {actor_reports[3]['name']}</button>
""")

In [ ]:
#@title 結果出力(Table)
mode = "total" #@param ["total", "actor_detail"]
target_actor = 0 #@param ["0", "1", "2", "3"] {type:"raw"}

#@markdown

#@markdown ####パラメータの説明
#@markdown mode: 出力方法
#@markdown - total: 全プレイヤーの最終結果を出力します
#@markdown - actor_detail: 対象のプレイヤーの局ごとの結果を出力します

#@markdown target_actor: (mode = 'actor_detail' の時のみ) 対象のプレイヤー

df = None
if mode == "total":
  df = create_total_result(actor_reports)
elif mode == "actor_detail":
  df = create_actor_result(actor_reports, target_actor)
df

In [ ]:
#@title 詳細レポート保存
!rm -rfd result | true
!rm artifacts.zip | true
!mkdir result
open("result/pred.json", "wt").write(json.dumps(report["pred"], indent=4))
open("result/meta.json", "wt").write(
    json.dumps({
        "playerInfo": report["playerInfo"],
        "haihuId": report["haihuId"],
        "nagaVersion": report["nagaVersion"],
        "gameType": report["gameType"]
    }, ensure_ascii = False, indent=4)
)

open("result/actor_0.json", "wt").write(json.dumps(actor_reports[0], ensure_ascii = False))
open("result/actor_1.json", "wt").write(json.dumps(actor_reports[1], ensure_ascii = False))
open("result/actor_2.json", "wt").write(json.dumps(actor_reports[2], ensure_ascii = False))
open("result/actor_3.json", "wt").write(json.dumps(actor_reports[3], ensure_ascii = False))

open("result/score.csv", "wt").write(create_total_result(actor_reports).to_csv())
open("result/actor_0_score.csv", "wt").write(create_actor_result(actor_reports, 0).to_csv())
open("result/actor_1_score.csv", "wt").write(create_actor_result(actor_reports, 1).to_csv())
open("result/actor_2_score.csv", "wt").write(create_actor_result(actor_reports, 2).to_csv())
open("result/actor_3_score.csv", "wt").write(create_actor_result(actor_reports, 3).to_csv())

!zip -j -r artifacts.zip result
files.download("artifacts.zip")